In [4]:
import pandas as pd
from fpdf import FPDF

# Function to generate a PDF for a single student
def generate_marksheet_pdf(student_data):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Add candidate details from the CSV row
    pdf.cell(200, 10, txt="Marksheet", ln=True, align="C")
    pdf.cell(200, 10, txt=f"Name: {student_data['Name']}", ln=True)
    pdf.cell(200, 10, txt=f"Roll No: {student_data['id']}", ln=True)
    # Add other details like Gender, Age, Section
    # ...

    pdf.cell(200, 10, txt="", ln=True) # Spacer

    # Add subject-wise marks in a table-like format
    pdf.set_font("Arial", 'B', size=12)
    pdf.cell(50, 10, "Subject", 1)
    pdf.cell(50, 10, "Obtained Marks", 1)
    pdf.ln()

    pdf.set_font("Arial", size=12)
    for subject in ['Science', 'English', 'History', 'Maths']:
        pdf.cell(50, 10, subject, 1)
        pdf.cell(50, 10, str(student_data[subject]), 1)
        pdf.ln()

    # Add overall result
    total_marks = student_data['Science'] + student_data['English'] + student_data['History'] + student_data['Maths']
    pdf.cell(200, 10, txt="", ln=True) # Spacer
    pdf.cell(200, 10, f"Total Marks: {total_marks}", ln=True)

    # Save the PDF
    pdf_filename = f"marksheet_{student_data['Name'].replace(' ', '_')}_{student_data['id']}.pdf"
    pdf.output(pdf_filename)
    print(f"Generated {pdf_filename}")

# Load your CSV data
df = pd.read_csv('/Users/tanishagangrade/Downloads/marksheet.csv')

# Iterate through the DataFrame and generate a PDF for each student
for index, row in df.iterrows():
    generate_marksheet_pdf(row)

Generated marksheet_Bronnie_1.pdf
Generated marksheet_Lemmie_2.pdf
Generated marksheet_Danya_3.pdf
Generated marksheet_Denna_4.pdf
Generated marksheet_Jocelin_5.pdf
Generated marksheet_Malissa_6.pdf
Generated marksheet_Ichabod_7.pdf
Generated marksheet_Beverlie_8.pdf
Generated marksheet_Corrine_9.pdf
Generated marksheet_Tate_10.pdf
Generated marksheet_Dunn_11.pdf
Generated marksheet_Valaria_12.pdf
Generated marksheet_Merrick_13.pdf
Generated marksheet_Gracie_14.pdf
Generated marksheet_Missy_15.pdf
Generated marksheet_Gilberta_16.pdf
Generated marksheet_Peggy_17.pdf
Generated marksheet_Dory_18.pdf
Generated marksheet_Opalina_19.pdf
Generated marksheet_Ashlan_20.pdf
Generated marksheet_Amery_21.pdf
Generated marksheet_Benoite_22.pdf
Generated marksheet_Merle_23.pdf
Generated marksheet_Valma_24.pdf
Generated marksheet_Jone_25.pdf
Generated marksheet_Rae_26.pdf
Generated marksheet_Charmine_27.pdf
Generated marksheet_Hanan_28.pdf
Generated marksheet_Adrian_29.pdf
Generated marksheet_Livia_3

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
import joblib
import json
from tqdm import tqdm
from pathlib import Path
import numpy as np
from PIL import Image
import random
import pytesseract

DATA_DIR = Path("marksheet_dataset")
IMAGES_DIR = DATA_DIR / "images"
LABELS_PATH = DATA_DIR / "labels.json"
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(exist_ok=True)

with open(LABELS_PATH) as f:
    labels = json.load(f)

# OCR helper
def image_to_full_text(img):
    data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    confs = [c for c in data['conf'] if c != -1]
    avg_conf = np.mean(confs) / 100 if confs else 0.8
    text = " ".join(data['text'])
    return text, avg_conf

# create features X and labels y
X, y = [], []
for key, entry in tqdm(labels.items()):
    png = entry.get("file_png")
    img_path = IMAGES_DIR / png
    if not img_path.exists():
        continue
    img = Image.open(img_path).convert("RGB")
    _, ocr_conf = image_to_full_text(img)

    # candidate_details positives and some negatives
    for field, val in entry.get("candidate_details", {}).items():
        # positive sample
        llm_conf = random.uniform(0.85, 0.99)
        feat = [ocr_conf, llm_conf, min(len(str(val or ""))/200, 1.0), 0]
        X.append(feat); y.append(1)
        # synthetic negative
        if random.random() < 0.25:
            wrong_llm = random.uniform(0.1, 0.5)
            feat = [ocr_conf, wrong_llm, random.random(), 1]
            X.append(feat); y.append(0)

X = np.array(X)
y = np.array(y)

print("Samples collected:", X.shape, y.shape)

# Train model if we have samples
if len(y) > 0:
    base = LogisticRegression(max_iter=2000)
    clf = CalibratedClassifierCV(base, cv=3)
    clf.fit(X, y)
    joblib.dump(clf, MODEL_PATH / "confidence_model.pkl")
    print("✅ Saved model to", MODEL_PATH / "confidence_model.pkl")
else:
    print("⚠️ No training samples found. Check dataset paths and parsing logic.")


0it [00:00, ?it/s]

Samples collected: (0,) (0,)
⚠️ No training samples found. Check dataset paths and parsing logic.


In [5]:
from pathlib import Path

IMAGES_DIR = Path("marksheet_dataset/images")
all_imgs = list(IMAGES_DIR.glob("*.png"))
print("Total images found:", len(all_imgs))
print([img.name for img in all_imgs[:5]])


Total images found: 2830
['marksheet_000548_v4.png', 'marksheet_000219_v4.png', 'marksheet_000177_v3.png', 'marksheet_000491_v3.png', 'marksheet_000077_v3.png']


In [7]:
import pandas as pd
import json
from pathlib import Path

# Paths
CSV_PATH = Path("/Users/tanishagangrade/Downloads/marksheet.csv")  # your original CSV
IMAGES_DIR = Path("marksheet_dataset/images")
LABELS_PATH = Path("marksheet_dataset/labels.json")

# Load CSV
df = pd.read_csv(CSV_PATH)

labels = {}

for idx, row in df.iterrows():
    key = f"student_{row['id']}"
    img_file = f"marksheet_{row['Name'].replace(' ', '_')}_{row['id']}.png"

    labels[key] = {
        "file_png": img_file,
        "candidate_details": {
            "Name": row["Name"],
            "Roll No": row["id"],
            "Science": row["Science"],
            "English": row["English"],
            "History": row["History"],
            "Maths": row["Maths"],
            "Total": row["Science"] + row["English"] + row["History"] + row["Maths"]
        }
    }

with open(LABELS_PATH, "w") as f:
    json.dump(labels, f, indent=2)

print(f"✅ labels.json created with {len(labels)} entries at {LABELS_PATH}")


✅ labels.json created with 250 entries at marksheet_dataset/labels.json


In [8]:
with open("marksheet_dataset/labels.json") as f:
    data = json.load(f)
print(list(data.items())[:2])  # preview first 2


[('student_1', {'file_png': 'marksheet_Bronnie_1.png', 'candidate_details': {'Name': 'Bronnie', 'Roll No': 1, 'Science': 21, 'English': 81, 'History': 62, 'Maths': 49, 'Total': 213}}), ('student_2', {'file_png': 'marksheet_Lemmie_2.png', 'candidate_details': {'Name': 'Lemmie', 'Roll No': 2, 'Science': 29, 'English': 41, 'History': 17, 'Maths': 40, 'Total': 127}})]


In [9]:
from pdf2image import convert_from_path
from pathlib import Path

PDF_DIR = Path("dataset/pdfs")    # adjust to where your PDFs are
IMG_DIR = Path("marksheet_dataset/images")
IMG_DIR.mkdir(parents=True, exist_ok=True)

for pdf_file in PDF_DIR.glob("*.pdf"):
    images = convert_from_path(pdf_file, dpi=200)
    img_path = IMG_DIR / (pdf_file.stem + ".png")
    images[0].save(img_path, "PNG")
    print(f"Saved {img_path}")


In [10]:
import json
from pathlib import Path

IMAGES_DIR = Path("marksheet_dataset/images")
LABELS_PATH = Path("marksheet_dataset/labels.json")

with open(LABELS_PATH) as f:
    labels = json.load(f)

missing = [entry["file_png"] for entry in labels.values()
           if not (IMAGES_DIR / entry["file_png"]).exists()]

print("Missing images:", missing[:10])
print("Total missing:", len(missing))


Missing images: ['marksheet_Bronnie_1.png', 'marksheet_Lemmie_2.png', 'marksheet_Danya_3.png', 'marksheet_Denna_4.png', 'marksheet_Jocelin_5.png', 'marksheet_Malissa_6.png', 'marksheet_Ichabod_7.png', 'marksheet_Beverlie_8.png', 'marksheet_Corrine_9.png', 'marksheet_Tate_10.png']
Total missing: 250


In [11]:
from pdf2image import convert_from_path
from pathlib import Path

PDF_DIR = Path("/Users/tanishagangrade/Downloads/marksheet_pdfs")  # folder where you stored all generated PDFs
IMG_DIR = Path("marksheet_dataset/images")
IMG_DIR.mkdir(parents=True, exist_ok=True)

for pdf_file in PDF_DIR.glob("*.pdf"):
    images = convert_from_path(pdf_file, dpi=200)  # convert PDF to images
    img_path = IMG_DIR / (pdf_file.stem + ".png")
    images[0].save(img_path, "PNG")
    print(f"✅ Saved {img_path}")


In [12]:
missing = [entry["file_png"] for entry in labels.values()
           if not (IMG_DIR / entry["file_png"]).exists()]
print("Total missing:", len(missing))


Total missing: 250


In [13]:
from pdf2image import convert_from_path
from pathlib import Path

# Adjust this path to where your PDFs are actually stored
PDF_DIR = Path("/Users/tanishagangrade/Downloads/marksheet_pdfs")
IMG_DIR = Path("marksheet_dataset/images")
IMG_DIR.mkdir(parents=True, exist_ok=True)

for pdf_file in PDF_DIR.glob("*.pdf"):
    images = convert_from_path(pdf_file, dpi=200)
    img_path = IMG_DIR / (pdf_file.stem + ".png")
    images[0].save(img_path, "PNG")
    print(f"✅ Saved {img_path}")

print("Conversion complete!")


Conversion complete!


In [14]:
missing = [entry["file_png"] for entry in labels.values()
           if not (IMG_DIR / entry["file_png"]).exists()]
print("Total missing:", len(missing))


Total missing: 250


In [16]:
from pathlib import Path

PDF_DIR = Path(".")   # current directory
all_pdfs = list(PDF_DIR.glob("marksheet_*.pdf"))
print("Found PDFs:", len(all_pdfs))
print([pdf.name for pdf in all_pdfs[:10]])


Found PDFs: 250
['marksheet_Reagan_159.pdf', 'marksheet_Ario_95.pdf', 'marksheet_Valaria_12.pdf', 'marksheet_Ichabod_7.pdf', 'marksheet_Ezekiel_190.pdf', 'marksheet_Valma_24.pdf', 'marksheet_Amery_21.pdf', 'marksheet_Reg_212.pdf', 'marksheet_Jacenta_45.pdf', 'marksheet_Malissa_6.pdf']


In [20]:
import fitz  # PyMuPDF
from pathlib import Path

PDF_DIR = Path(".")  # current directory
IMG_DIR = Path("marksheet_dataset/images")
IMG_DIR.mkdir(parents=True, exist_ok=True)

for pdf_file in PDF_DIR.glob("marksheet_*.pdf"):
    doc = fitz.open(pdf_file)
    page = doc[0]  # first page
    pix = page.get_pixmap(dpi=200)
    img_path = IMG_DIR / (pdf_file.stem + ".png")
    pix.save(img_path)
    print(f"✅ Saved {img_path}")


✅ Saved marksheet_dataset/images/marksheet_Reagan_159.png
✅ Saved marksheet_dataset/images/marksheet_Ario_95.png
✅ Saved marksheet_dataset/images/marksheet_Valaria_12.png
✅ Saved marksheet_dataset/images/marksheet_Ichabod_7.png
✅ Saved marksheet_dataset/images/marksheet_Ezekiel_190.png
✅ Saved marksheet_dataset/images/marksheet_Valma_24.png
✅ Saved marksheet_dataset/images/marksheet_Amery_21.png
✅ Saved marksheet_dataset/images/marksheet_Reg_212.png
✅ Saved marksheet_dataset/images/marksheet_Jacenta_45.png
✅ Saved marksheet_dataset/images/marksheet_Malissa_6.png
✅ Saved marksheet_dataset/images/marksheet_Raimund_240.png
✅ Saved marksheet_dataset/images/marksheet_Rosamond_173.png
✅ Saved marksheet_dataset/images/marksheet_Rina_107.png
✅ Saved marksheet_dataset/images/marksheet_Jone_25.png
✅ Saved marksheet_dataset/images/marksheet_Rosalyn_206.png
✅ Saved marksheet_dataset/images/marksheet_Goldarina_187.png
✅ Saved marksheet_dataset/images/marksheet_Benoite_22.png
✅ Saved marksheet_datas

In [21]:
import json
from pathlib import Path

IMG_DIR = Path("marksheet_dataset/images")
with open("marksheet_dataset/labels.json") as f:
    labels = json.load(f)

missing = [entry["file_png"] for entry in labels.values()
           if not (IMG_DIR / entry["file_png"]).exists()]
print("Total missing:", len(missing))


Total missing: 0


In [22]:
import random
import numpy as np
from PIL import Image
from tqdm import tqdm

# Dummy OCR function (replace later with actual OCR if needed)
def image_to_full_text(img):
    # return text, confidence (simulate OCR confidence 0.6-0.95)
    return "dummy text", random.uniform(0.6, 0.95)

X, y = [], []

for key, entry in tqdm(labels.items()):
    img_path = IMG_DIR / entry["file_png"]
    if not img_path.exists():
        continue

    img = Image.open(img_path).convert("RGB")
    _, ocr_conf = image_to_full_text(img)

    # candidate_details positives + negatives
    for field, val in entry.get("candidate_details", {}).items():
        # positive sample
        llm_conf = random.uniform(0.85, 0.99)
        feat = [ocr_conf, llm_conf, min(len(str(val or "")) / 200, 1.0), 0]
        X.append(feat)
        y.append(1)

        # negative synthetic sample
        if random.random() < 0.25:
            wrong_llm = random.uniform(0.1, 0.5)
            feat = [ocr_conf, wrong_llm, random.random(), 1]
            X.append(feat)
            y.append(0)

X = np.array(X)
y = np.array(y)

print("Samples collected:", X.shape, y.shape)


100%|█████████████████████████████████████████| 250/250 [00:08<00:00, 29.14it/s]

Samples collected: (2188, 4) (2188,)


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
import joblib
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(exist_ok=True)

base = LogisticRegression(max_iter=2000)
clf = CalibratedClassifierCV(base, cv=3)
clf.fit(X, y)

joblib.dump(clf, MODEL_PATH / "confidence_model.pkl")
print("✅ Model trained and saved to", MODEL_PATH / "confidence_model.pkl")


✅ Model trained and saved to models/confidence_model.pkl


In [24]:
sample = X[0].reshape(1, -1)
print("Prediction:", clf.predict(sample))
print("Confidence:", clf.predict_proba(sample))


Prediction: [1]
Confidence: [[0.00154217 0.99845783]]


In [25]:
import joblib
from pathlib import Path

MODEL_PATH = Path("/Users/tanishagangrade/Desktop/MarkSheet_ConfidenceScore/models")
MODEL_PATH.mkdir(exist_ok=True)

joblib.dump(clf, MODEL_PATH / "confidence_model.pkl")
print("Model saved!")


Model saved!
